In [1]:
# demo.ipynb

# ======================
# 1. 环境与路径设置
# ======================
import os
import argparse
import pandas as pd
import matplotlib.pyplot as plt

from train import train
from eval import evaluate
from geometry_baseline import run_baseline

DATA_ROOT = "/Users/zhengbinheng/Desktop/7-scenes-dataset"   # 修改成你的数据集路径
OUT_DIR = "runs/posenet_chess"
CKPT_PATH = os.path.join(OUT_DIR, "best.ckpt")
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)


In [2]:
# ======================
# 2. 训练 PoseNet (Chess)
# ======================
args = argparse.Namespace(
    data_root=DATA_ROOT,
    scene="chess",
    epochs=30,
    batch_size=32,
    lr=1e-4,
    beta=120.0,
    out=OUT_DIR
)
train(args)  # 会生成 best.ckpt + loss 曲线


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# ======================
# 3. 可视化训练曲线
# ======================
df_loss = pd.read_csv(os.path.join(OUT_DIR, "loss_curve.csv"))
plt.plot(df_loss["epoch"], df_loss["loss"], marker="o")
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("PoseNet Training Loss (Chess)")
plt.grid(True)
plt.show()


In [ ]:
# ======================
# 4. PoseNet 测试与泛化
# ======================
def run_eval(scene):
    args = argparse.Namespace(
        data_root=DATA_ROOT,
        scene=scene,
        ckpt=CKPT_PATH,
        out=os.path.join(RESULTS_DIR, f"{scene}_posenet.csv")
    )
    stats, df = evaluate(args)
    return stats, df

scenes = ["chess", "pumpkin", "redkitchen"]

pose_stats, pose_perframe = {}, {}
for s in scenes:
    stats, df = run_eval(s)
    pose_stats[s] = stats
    pose_perframe[s] = df


In [ ]:
# ======================
# 5. PnP+RANSAC 基线
# ======================
def run_baseline_eval(scene):
    args = argparse.Namespace(
        data_root=DATA_ROOT,
        scene=scene,
        orb_kpts=2000,
        topk=5
    )
    stats, df = run_baseline(args)
    return stats, df

base_stats, base_perframe = {}, {}
for s in scenes:
    stats, df = run_baseline_eval(s)
    base_stats[s] = stats
    base_perframe[s] = df


In [ ]:
# ======================
# 6. 汇总对比表格
# ======================
summary = pd.DataFrame([
    {
        "Scene": s,
        "PoseNet mean_t (m)": pose_stats[s]["mean_t"],
        "PoseNet median_t (m)": pose_stats[s]["median_t"],
        "PoseNet mean_r (°)": pose_stats[s]["mean_r"],
        "PoseNet median_r (°)": pose_stats[s]["median_r"],
        "PnP+RANSAC mean_t (m)": base_stats[s]["mean_t"],
        "PnP+RANSAC median_t (m)": base_stats[s]["median_t"],
        "PnP+RANSAC mean_r (°)": base_stats[s]["mean_r"],
        "PnP+RANSAC median_r (°)": base_stats[s]["median_r"],
    }
    for s in scenes
])

summary.round(3)


In [ ]:
# ======================
# 7. 误差分布可视化 (直方图)
# ======================
for s in scenes:
    df_pose = pose_perframe[s]
    df_base = base_perframe[s]

    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    plt.hist(df_pose["t_err_m"], bins=30, alpha=0.6, label="PoseNet")
    plt.hist(df_base["t_err_m"], bins=30, alpha=0.6, label="PnP+RANSAC")
    plt.xlabel("Translation Error (m)")
    plt.ylabel("Count")
    plt.title(f"{s} - Translation Error")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.hist(df_pose["r_err_deg"], bins=30, alpha=0.6, label="PoseNet")
    plt.hist(df_base["r_err_deg"], bins=30, alpha=0.6, label="PnP+RANSAC")
    plt.xlabel("Rotation Error (°)")
    plt.ylabel("Count")
    plt.title(f"{s} - Rotation Error")
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
# ======================
# 8. 导出 LaTeX 表格 (可选)
# ======================
latex_table = summary.round(3).to_latex(index=False)
with open(os.path.join(RESULTS_DIR, "summary_table.tex"), "w") as f:
    f.write(latex_table)

print("LaTeX table saved to results/summary_table.tex")


In [ ]:
# ======================
# 9. 轨迹可视化 (GT vs PoseNet vs PnP+RANSAC)
# ======================
from mpl_toolkits.mplot3d import Axes3D

def plot_trajectory(scene):
    # 读取逐帧结果
    df_pose = pd.read_csv(f"results/{scene}_posenet_perframe.csv")
    df_base = pd.read_csv(f"results/{scene}_baseline_perframe.csv")

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection="3d")

    # GT
    ax.plot(df_pose["t_gt_x"], df_pose["t_gt_y"], df_pose["t_gt_z"],
            "k-", label="Ground Truth")

    # PoseNet
    ax.plot(df_pose["t_pred_x"], df_pose["t_pred_y"], df_pose["t_pred_z"],
            "r--", label="PoseNet")

    # Baseline
    ax.plot(df_base["t_pred_x"], df_base["t_pred_y"], df_base["t_pred_z"],
            "b--", label="PnP+RANSAC")

    ax.set_title(f"Camera Trajectory - {scene}")
    ax.set_xlabel("X (m)")
    ax.set_ylabel("Y (m)")
    ax.set_zlabel("Z (m)")
    ax.legend()
    plt.show()


# 示例：画 chess 场景
plot_trajectory("chess")


In [ ]:
# 可选：2D 投影轨迹 (X-Z 平面)

def plot_trajectory_2d(scene):
    df_pose = pd.read_csv(f"results/{scene}_posenet_perframe.csv")
    df_base = pd.read_csv(f"results/{scene}_baseline_perframe.csv")

    plt.figure(figsize=(8,6))

    # GT
    plt.plot(df_pose["t_gt_x"], df_pose["t_gt_z"], "k-", label="Ground Truth")

    # PoseNet
    plt.plot(df_pose["t_pred_x"], df_pose["t_pred_z"], "r--", label="PoseNet")

    # Baseline
    plt.plot(df_base["t_pred_x"], df_base["t_pred_z"], "b--", label="PnP+RANSAC")

    plt.title(f"Trajectory (X-Z projection) - {scene}")
    plt.xlabel("X (m)")
    plt.ylabel("Z (m)")
    plt.legend()
    plt.grid(True)
    plt.show()


plot_trajectory_2d("chess")
